<a href="https://colab.research.google.com/github/vivekzzz55/firstproject/blob/main/stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense


In [ ]:
data = pd.read_csv('TSLA.csv', date_parser=True)
data = data[['Date', 'Close']]
data = data.sort_values('Date')

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
data['Close'] = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

In [ ]:
def create_sequences(data, seq_length):
    sequences, labels = [], []
    for i in range(len(data) - seq_length):
        seq = data['Close'].values[i:i+seq_length]
        label = data['Close'].values[i+seq_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [ ]:
sequence_length = 10
X, y = create_sequences(data, sequence_length)


In [ ]:
train_size = int(len(X) * 0.80)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32)


In [ ]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)


In [ ]:
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
plt.figure(figsize=(30, 8))
plt.plot(data['Date'][-len(y_test):], y_test, label='Actual Prices')
plt.plot(data['Date'][-len(y_test):], predictions, label='Predicted Prices')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.title('Stock Price Prediction Using LSTM')
plt.legend()
plt.show()